In [3]:
import csv

import pandas as pd
from glob import glob

file_total_original = "data/hate_speech_hindi_english.tsv"
df_total = pd.read_csv(file_total_original,sep="\t",encoding='utf-8',quoting=csv.QUOTE_NONE,usecols=[0,1])
df_total = df_total.dropna()

total_sentences = list(df_total['text'].values)
total_labels = list(df_total['label'].values)

fin = open("sentences.txt","w+",encoding="utf-8")

for line in total_sentences:
    fin.write(line+"\n")

length = []
for line in total_sentences:
    line_list = line.split()
    length.append(len(line_list))

from collections import Counter

#print(Counter(length).most_common())

max_length = 2*Counter(length).most_common()[-1][0]
print(max_length)

134


In [4]:
from pathlib import Path
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files="sentences.txt", vocab_size=52_000, min_frequency=2, special_tokens=[
    "<s>",
    "<pad>",
    "</s>",
    "<unk>",
    "<mask>",
])

In [5]:
!mkdir CM_bert
tokenizer.save_model("CM_bert")

['CM_bert\\vocab.json', 'CM_bert\\merges.txt']

In [6]:
from tokenizers.implementations import ByteLevelBPETokenizer
from tokenizers.processors import BertProcessing

tokenizer = ByteLevelBPETokenizer(
    "./CM_bert/vocab.json",
    "./CM_bert/merges.txt",
)

In [7]:
tokenizer._tokenizer.post_processor = BertProcessing(
    ("</s>", tokenizer.token_to_id("</s>")),
    ("<s>", tokenizer.token_to_id("<s>")),
)
tokenizer.enable_truncation(max_length=max_length)

In [8]:
tokenizer.encode(total_sentences[0]).tokens

['<s>',
 'K',
 'now',
 'ing',
 'Ġki',
 'ĠVikas',
 'Ġkitna',
 'Ġsamjha',
 'ta',
 'Ġhai',
 'ĠP',
 'riyanka',
 'Ġaur',
 'ĠIt',
 'ch',
 'ĠGu',
 'ard',
 'ĠLuv',
 'Ġko',
 ',',
 'Ġusne',
 'Ġbola',
 'Ġtha',
 'ĠB',
 'en',
 'Ġwali',
 'Ġbaat',
 'Ġme',
 'Ġab',
 'ĠSallu',
 'Ġne',
 'Ġbhi',
 'Ġagree',
 'Ġkiya',
 '!',
 '</s>']

In [9]:
import torch
torch.cuda.is_available()

True

In [10]:
from transformers import RobertaConfig

config = RobertaConfig(
    vocab_size=52_000,
    max_position_embeddings=max_length,
    num_attention_heads=3,
    num_hidden_layers=3,
    type_vocab_size=1,
)

In [11]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("./CM_bert", max_len=max_length)

In [12]:
from transformers import RobertaForMaskedLM

model = RobertaForMaskedLM(config=config)

model.num_parameters()

61948960

In [13]:
from transformers import LineByLineTextDataset

dataset = LineByLineTextDataset(
    tokenizer=tokenizer,
    file_path="sentences.txt",
    block_size=8,
)

C:\Users\suman\Miniconda3\envs\py3_env\lib\site-packages\transformers\data\datasets\language_modeling.py:128: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/master/examples/language-modeling/run_mlm.py
  FutureWarning,


In [14]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.01
)

In [15]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./CM_bert",
    overwrite_output_dir=True,
    num_train_epochs=6,
    per_device_train_batch_size=1,
    save_steps=10_000,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=dataset,
)

trainer.train()

Step,Training Loss
500,0.613300
1000,0.605000
1500,0.760900
2000,0.490300
2500,0.600800
3000,0.554300
3500,0.644800
4000,0.679100
4500,0.561300
5000,0.708500


TrainOutput(global_step=27468, training_loss=0.5511127014737888, metrics={'train_runtime': 722.1708, 'train_samples_per_second': 38.035, 'total_flos': 81483449454720, 'epoch': 6.0})

In [16]:
trainer.save_model("./CM_bert")